In [1]:
import os
import numpy as np
import pandas as pd
import dask.dataframe as dd

C:\ProgramData\Anaconda3\lib\site-packages\dask\dataframe\utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
from preprocessing import preprocess

In [3]:
%%time
df = pd.read_excel('indeed_results_new.xlsx')
for n, file in enumerate(os.listdir('data/')):
    if file.endswith('.xlsx'):
        print(f'{n+1}, {file}')
        df = df.append(pd.read_excel('data/'+file), ignore_index=True)
        df.drop_duplicates().to_excel('indeed_results_new.xlsx', index=False)
        os.remove('data/'+file)

Wall time: 1.55 s


In [4]:
df = pd.read_excel('indeed_results_new.xlsx')

In [5]:
df

,Title,Location,Company,Salary,Sponsored,Description,Time
0,PGT Art 'N' Craft,"Jhumri Tilaiya, Jharkhand",GRIZZLY VIDYALAYA,"₹22,000 - ₹25,000 a month",Sponsored,Job Summary\nGRIZZLY VIDYALAYA : An ISO 9001:2...,2020-02-10
1,TGT BIOLOGY,"Jhumri Tilaiya, Jharkhand",GRIZZLY VIDYALAYA,"₹22,000 - ₹28,000 a month",Sponsored,Job Summary\nGRIZZLY VIDYALAYA : An ISO 9001:2...,2020-02-22
2,TGT COMPUTER 6-8,"Jhumri Tilaiya, Jharkhand",GRIZZLY VIDYALAYA,"₹16,000 - ₹20,000 a month",Sponsored,Job Summary\nGRIZZLY VIDYALAYA : An ISO 9001:2...,2020-02-22
3,Fashion Design faculty,"Chinchwad, Pune, Maharashtra",Times and Trends Academy Chinchwad,"₹20,000 a month",Sponsored,Job Summary\nIf you are interested in a high-g...,2020-02-17
4,Data Entry Back Office,"Gwalior, Madhya Pradesh",World Collection,"₹45,000 a month",Organic,"Locality- Birla Nagar, Kailash Nagar, Malanpur...",2020-02-25
...,...,...,...,...,...,...,...
17133,Accounts Executive,Pune,DVC Process Technologists,"₹15,000 - ₹20,000 a month",Organic,Job Summary\nAccounts Payable including bill b...,2020-03-04
17134,Club Lounge Agent,Pune,"Marriott International, Inc",NaN,Organic,"Posting Date Mar 02, 2020\nJob Number 20028713...",2020-03-04
17135,Customer Assistant,Pune,credence resource management,"₹20,668 - ₹35,800 a year",Organic,Greeting of the day\nwe are hiring urgently fo...,2020-02-25
17136,Inside Sales Executive,Pune,The Learners Camp,"₹4,00,000 a year",Organic,We sell high-quality online courses on a varie...,2020-02-28


In [65]:
df.dropna(inplace=True)

In [66]:
df['Description'].unique()[np.random.randint(0, df['Description'].nunique())]

' We are Hiring E-Commerce Executive For All E-Commerce Portal Like amazon, flipkart, club factory. Responsibilities and Duties 1.To manage seller account 2. Listing 3. Sales Boost Key Skills Knowledge excl and basic computer Required Experience and Qualifications fresher also apply Job Type: Full-time Salary: ₹8,000.00 to ₹20,000.00 /month Experience: work: 1 year (Preferred) total work: 1 year (Preferred) Education: Higher Secondary(12th Pass) (Preferred) Location: Delhi, Delhi (Preferred) Benefits: None Industry: Administrative Assistance'

In [73]:
def preprocessing_title_location_company(text):
    removal_notations = ["'", '"', '(', ')', '[', ']', '-', '/', '.', ',', '`', ':', '|', '\\', ';', '<', '>', '?', '~', '!', '#', '$', '%', '^', '*', '_', '+', '=', '{', '}']
    for removal_notation in removal_notations:
        for _ in range(20):
            text.replace(removal_notation, ' ')
    for _ in range(10):
        text = text.replace('&', 'and')
        text = text.replace('\'', ' ')
    for _ in range(10):
        text = text.replace("  ", ' ')
    return text

In [188]:
def preprocessing_description(text):
    for _ in range(100):
        text = text.replace('\n', ' ')
    for _ in range(10):
        text = text.replace('&', ' and ')
        text = text.replace('·', ' ')
        text = text.replace('/', ' ')
        text = text.replace('(', ' ')
        text = text.replace(')', ' ')
        text = text.replace('[', ' ')
        text = text.replace(']', ' ')
        text = text.replace(':', ' ')
        text = text.replace('.', ' ')
        text = text.replace('-', ' ')
        text = text.replace('#', ' ')
        text = text.replace("'", ' ')
        text = text.replace('"', ' ')
        text = text.replace('`', ' ')
        text = text.replace('|', ' ')
        text = text.replace(';', ' ')
        text = text.replace('<', ' ')
        text = text.replace('>', ' ')
        text = text.replace('?', ' ')
        text = text.replace('~', ' ')
        text = text.replace('!', ' ')
        text = text.replace('$', ' ')
        text = text.replace('%', ' ')
        text = text.replace('^', ' ')
        text = text.replace('*', ' ')
        text = text.replace('_', ' ')
        text = text.replace('+', ' ')
        text = text.replace('=', ' ')
        text = text.replace('{', ' ')
        text = text.replace('}', ' ')
        text = text.replace('–', ' ')
        text = text.replace('\\', ' ')
        text = text.replace(',', '')
    for _ in range(10):
        text = text.replace("  ", ' ')
    text = text.replace(r'\uf0b7', ' ')
    text = text.replace('\uf0b7', ' ')
    text = text.replace('Job Summary ', ' ')
    text = text.replace('Job Description ', ' ')
    return text

In [189]:
def preprocessing_salary(text):
    removal_notations = ["'", '"', '[', ']', '-', '.', '`', '|', '\\', ';', '<', '>', '?', '~', '!', '#', '$', '%', '^', '*', '_', '+', '=', '{', '}']
    for removal_notation in removal_notations:
        for _ in range(20):
            text.replace(removal_notation, ' ')
    for _ in range(20):
        text = text.replace('&', 'and')
        text = text.replace(',', '')
        text = text.replace('/', ' ')
        text = text.replace(':', ' ')
        text = text.replace('(', ' ')
        text = text.replace(')', ' ')
        
    for _ in range(5):
        text = text.replace('₹', '')
        text = text.replace('-', '')
        text = text.replace(' a ', ' ')
        text = text.replace(' an ', ' ')
    
    for _ in range(10):
        text = text.replace('  ', ' ')
    return text

In [190]:
%%time
df['Title'] = df['Title'].apply(lambda x: preprocessing_title_location_company(x))
df['Location'] = df['Location'].apply(lambda x: preprocessing_title_location_company(x))
df['Company'] = df['Company'].apply(lambda x: preprocessing_title_location_company(x))
df['Description'] = df['Description'].apply(lambda x: preprocessing_description(x))
df['Salary'] = df['Salary'].apply(lambda x: preprocessing_salary(x))

Wall time: 3.14 s


In [215]:
df['Description'].unique()[np.random.randint(0, df['Description'].nunique())]

'We are providing coaching for General competitive examination SSC BANK RAILWAY POLICE TET DSSSB TGT PGT PRT and others and going to start Academic session also So we are required good trained faculty dedicated and for our institute Job Types Part time Temporary Salary ₹5000 00 to ₹8000 00 month Experience work 1 year Preferred Teaching 1 year Preferred total work 1 year Preferred Education Bachelor s Preferred Shifts Morning Preferred Evening Preferred Required Documents Aadhar Card Preferred '

In [232]:
df['Salary'].unique()[np.random.randint(0, df['Salary'].nunique())]

'5000 15000 month'